In [1]:
import sys
sys.path.insert(0,"/p/home/jusers/schmidt9/juwels/PROJECT_testufz/Custompip/lib/python3.6/site-packages")

In [15]:
tf.reset_default_graph()

In [2]:
import os,time,cv2, sys, math
import tensorflow as tf
import argparse
import numpy as np

from utils import utils, helpers
from builders import model_builder

dataset= "data" #dataset-folder, default:"CamVid"
crop_height=512 #Height of cropped input image to network
crop_width=512 #Width of cropped input image to network
#model="Encoder-Decoder" #The model you are using. See model_builder.py for supported models,"FC-DenseNet56"
model="MobileUNet"
checkpoint_path="checkpoints/latest_model_" +  model + "_" +  dataset + ".ckpt"
#image = "data/test/2019_0821_145335_114.jpg"
imagefolder="data/test"
images=[]
import os
for file in os.listdir(imagefolder):
    if file.endswith(".jpg"):
        images.append(os.path.join(imagefolder, file))

class_names_list, label_values = helpers.get_label_info(os.path.join( dataset, "class_dict.csv"))

num_classes = len(label_values)

print("\n***** Begin prediction *****")
print("Dataset -->",  dataset)
print("Model -->",  model)
print("Crop Height -->",  crop_height)
print("Crop Width -->",  crop_width)
print("Num Classes -->", num_classes)
#print("Image -->",  image)
tf.reset_default_graph()#delete previous variables (?)

# Initializing network
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess=tf.Session(config=config)

net_input = tf.placeholder(tf.float32,shape=[None,None,None,3])
net_output = tf.placeholder(tf.float32,shape=[None,None,None,num_classes]) 

network, _ = model_builder.build_model( model, net_input=net_input,
                                        num_classes=num_classes,
                                        crop_width= crop_width,
                                        crop_height= crop_height,
                                        is_training=False)

sess.run(tf.global_variables_initializer())

print('Loading model checkpoint weights')
saver=tf.train.Saver(max_to_keep=1000)
saver.restore(sess,  checkpoint_path)

for image in images:
    print("Testing image " +  image)

    loaded_image = utils.load_image(image)
    resized_image =cv2.resize(loaded_image, ( crop_width,  crop_height))
    input_image = np.expand_dims(np.float32(resized_image[: crop_height, : crop_width]),axis=0)/255.0

    st = time.time()
    output_image = sess.run(network,feed_dict={net_input:input_image})

    run_time = time.time()-st

    output_image = np.array(output_image[0,:,:,:])
    output_image = helpers.reverse_one_hot(output_image)

    out_vis_image = helpers.colour_code_segmentation(output_image, label_values)
    out_vis_image[:,:,0] = out_vis_image[:,:,0]*20
    out_vis_image[:,:,1] = out_vis_image[:,:,1]*10
    out_vis_image[:,:,2] = out_vis_image[:,:,2]*5
    # overlay input+predictions
    out_both = cv2.addWeighted(resized_image, 0.5, out_vis_image, 0.5, 0.0,dtype=64)
    file_name = utils.filepath_to_name(image)
    cv2.imwrite("%s_pred.png"%(file_name),cv2.cvtColor(np.uint8(out_both), cv2.COLOR_RGB2BGR))

    print("")
    print("Finished!")
    print("Wrote image " + "%s_pred.png"%(file_name))

sess.close()

/gpfs/software/juwels/stages/Devel-2019a/software/TensorFlow/1.13.1-GCCcore-8.3.0-GPU-Python-3.6.8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/gpfs/software/juwels/stages/Devel-2019a/software/TensorFlow/1.13.1-GCCcore-8.3.0-GPU-Python-3.6.8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/gpfs/software/juwels/stages/Devel-2019a/software/TensorFlow/1.13.1-GCCcore-8.3.0-GPU-Python-3.6.8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is


***** Begin prediction *****
Dataset --> data
Model --> MobileUNet
Crop Height --> 512
Crop Width --> 512
Num Classes --> 11
Preparing the model ...
Instructions for updating:
Colocations handled automatically by placer.
Loading model checkpoint weights
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from checkpoints/latest_model_MobileUNet_data.ckpt
Testing image data/test/2019_0821_145356_115.jpg

Finished!
Wrote image 2019_0821_145356_115_pred.png
Testing image data/test/2019_0821_145456_118.jpg

Finished!
Wrote image 2019_0821_145456_118_pred.png
Testing image data/test/2019_0821_145658_124.jpg

Finished!
Wrote image 2019_0821_145658_124_pred.png
Testing image data/test/2019_0821_145718_125.jpg

Finished!
Wrote image 2019_0821_145718_125_pred.png
Testing image data/test/2019_0821_145335_114.jpg

Finished!
Wrote image 2019_0821_145335_114_pred.png
Testing image data/test/2019_0821_145557_121.jpg

Finished!


In [13]:
print(out_vis_image.
    dtype)
print(resized_image.dtype)

int64
uint8
